##Loading the Libraries

In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string

##Reading the data

In [2]:
#reading the data
data = pd.read_csv("/content/drive/MyDrive/Exit test/labeledTrainData.tsv",sep='\t')

In [3]:
data

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [4]:
#dropping the id column
data.drop('id', axis=1, inplace=True)
data

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...
...,...,...
24995,0,It seems like more consideration has gone into...
24996,0,I don't believe they made this film. Completel...
24997,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,0,This 30 minute documentary Buñuel made in the ...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  25000 non-null  int64 
 1   review     25000 non-null  object
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [6]:
#checking null values
data.isna().sum()

sentiment    0
review       0
dtype: int64

In [7]:
#checking the shape of the dataset
data.shape

(25000, 2)

In [8]:
data.describe()

,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


In [9]:
data['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

##NLP Preprocessing

###Lowercasing

In [10]:
data["review"] =data["review"].astype(str)

In [11]:
data["review"] = data["review"].str.lower()
data.head()

,sentiment,review
0,1,with all this stuff going down at the moment w...
1,1,"\the classic war of the worlds\"" by timothy hi..."
2,0,the film starts with a manager (nicholas bell)...
3,0,it must be assumed that those who praised this...
4,1,superbly trashy and wondrously unpretentious 8...


###Removing Punctuation

In [12]:
import string
string.punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct


data["review"] = data["review"].apply(lambda x: remove_punct(x))

In [13]:
data

,sentiment,review
0,1,with all this stuff going down at the moment w...
1,1,the classic war of the worlds by timothy hines...
2,0,the film starts with a manager nicholas bell g...
3,0,it must be assumed that those who praised this...
4,1,superbly trashy and wondrously unpretentious 8...
...,...,...
24995,0,it seems like more consideration has gone into...
24996,0,i dont believe they made this film completely ...
24997,0,guy is a loser cant get girls needs to build u...
24998,0,this 30 minute documentary buñuel made in the ...


###Tokenization

In [14]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

data['review'] = data['review'].apply(lambda x: tokenize(x.lower()))


data.head()

,sentiment,review
0,1,"[with, all, this, stuff, going, down, at, the,..."
1,1,"[the, classic, war, of, the, worlds, by, timot..."
2,0,"[the, film, starts, with, a, manager, nicholas..."
3,0,"[it, must, be, assumed, that, those, who, prai..."
4,1,"[superbly, trashy, and, wondrously, unpretenti..."


###Remove Stopwords

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


data['review'] = data['review'].apply(lambda text: remove_stopwords(text))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


###Lemmatization

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])



data["review"] = data["review"].apply(lambda text: lemmatize_words(text))

In [18]:
#importing libraries

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [19]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

In [20]:
X_train = X_train.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
X_test = X_test.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [21]:
# Now you can proceed with the vectorization code
vectorizer_tfidf = TfidfVectorizer()
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

vectorizer_cvec = CountVectorizer()
X_train_cvec = vectorizer_cvec.fit_transform(X_train)
X_test_cvec = vectorizer_cvec.transform(X_test)

###ML Alogorithms

In [22]:
# Step 3: Define the machine learning algorithms
algorithms = {
    'Multinomial Naive Bayes': MultinomialNB(),
    'Random Forest': RandomForestClassifier(),
    #'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'SGC_Clsiifier': SGDClassifier(),
    'extratree_classifier' : ExtraTreesClassifier(),

}

In [23]:
# Train and evaluate each algorithm using Bag of Words
results_cvec = {}
for algorithm_name, algorithm in algorithms.items():
    algorithm.fit(X_train_cvec, y_train)
    y_pred = algorithm.predict(X_test_cvec)
    accuracy_cvec = accuracy_score(y_test, y_pred)
    results_cvec[algorithm_name] = accuracy_cvec

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
# Step 5: Train and evaluate each algorithm using TF-IDF
results_tfidf = {}
for algorithm_name, algorithm in algorithms.items():
    algorithm.fit(X_train_tfidf, y_train)
    y_pred = algorithm.predict(X_test_tfidf)
    accuracy_tfidf = accuracy_score(y_test, y_pred)
    results_tfidf[algorithm_name] = accuracy_tfidf

In [25]:
# Step 6: Create a DataFrame to display the results
results_df = pd.DataFrame({'Bag of Words': results_cvec, 'TF-IDF': results_tfidf})

# Display the results table
print(results_df)


                         Bag of Words  TF-IDF
Multinomial Naive Bayes        0.8486  0.8612
Random Forest                  0.8484  0.8368
Logistic Regression            0.8816  0.8914
SGC_Clsiifier                  0.8788  0.8964
extratree_classifier           0.8634  0.8562


Maximum accuracy is for Logistic regression which is 89%

###LSTM

In [1]:
from tensorflow.keras.preprocessing import sequence

In [ ]:
max_len=300

In [ ]:

X_train_tfidf_padded=sequence.pad_sequences(X_train_tfidf,maxlen= max_len)
X_test_tfidf_padded=sequence.pad_sequences(X_test_tfidf,maxlen= max_len)
print(X_train_tfidf_padded)
print(X_test_tfidf_padded)

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding

In [ ]:
model2=Sequential()
model2.add(Embedding(input_dim=20000, output_dim=150 , input_length=300))
model2.add(LSTM(128, dropout=.2))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

In [ ]:
model2.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
print('Training starts')
model2.fit(X_train_tfidf, y_train, batch_size=256, epochs=4)

In [ ]:
model2.evaluate(X_test_tfidf, y_test)